In [1]:
from pathlib import Path

import util
from util import p_, workflow

import automech

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [2]:
# Generate submechanism
from automol.graph import enum

from automech.species import Species

gen_mech = automech.from_smiles(
    spc_smis=["C12C(O2)[CH]CC1", "C12C(O2)C[CH]C1"],
    src_mech=par_mech,
)
#  - Additions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.o2_addition,
    rcts_=[None, "[O][O]"],
    spc_col_=Species.smiles,
    src_mech=par_mech,
)
#  - Eliminations
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.ho2_elimination,
    spc_col_=Species.smiles,
    src_mech=par_mech,
)
#  - Drop reactions:
gen_mech = automech.drop_reactions_by_smiles(
    # >> Beta elimination: suspiciously low frequency, could not be confirmed by IRC
    gen_mech,
    rxn_smis=["[O]OC1CCC2C1O2>>[O]O.C=1CCC2C=1O2"],
)
automech.display(gen_mech)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Expand stereochemistry
ste_mech, *_, gen_mech = workflow.expand_stereo(
    mech=gen_mech, tag=tag, root_path=root_path
)


Expanding stereochemistry...


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]


Stereoexpansion errors:


In [4]:
# Prepare calculation
workflow.prepare_calculation(
    gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
)


Sorting mechanism...

Writing mechanism...
../data/D_r-o2_2x_v0_gen.json
../data/D_r-o2_2x_v0_ste.json
../data/mechanalyzer/D_r-o2_2x_v0_ste.dat
../data/mechanalyzer/D_r-o2_2x_v0_ste.csv


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
# # Gather statistics
# workflow.gather_statistics(tag=tag, root_path=root_path)

In [6]:
# # Prepare simulation
# workflow.prepare_simulation(tag=tag, root_path=root_path)
# workflow.prepare_simulation_species(tag=tag, root_path=root_path)

In [7]:
# # Check rates
# workflow.plot_rates(tag=tag, root_path=root_path)

In [8]:
# # Plot simulation
# chart_dct = workflow.plot_simulation(
#     tag=tag,
#     x_col="O2_molecules",
#     root_path=root_path,
#     line_source_=["Hill", "Lokachari"],
#     point_source="Experiment",
#     control=False,
# )
# mech = automech.io.read(
#     p_.full_calculated_mechanism(tag, "json", path=p_.data(root_path))
# )
# for name, chart in chart_dct.items():
#     automech.display_species(mech, [name])
#     chart.show()

In [9]:
# # Check TS
# from mechdriver.subtasks import display

# chan = "2: 5"

# #   - Read in expanded mechanism
# mech_path = p_.mechanism(tag, ext="json", path=p_.data(root_path))
# mech = automech.io.read(mech_path)

# #   - Display the reaction
# automech.display_reactions(mech, chans=[chan])

In [10]:
# #   - Display the TS mode
# calc_path = p_.calc(root_path, tag)
# display("conf_opt", chan, path=calc_path)

In [11]:
# #   - Display the TS mode
# calc_path = p_.calc(root_path, tag)
# display("find_ts", chan, path=calc_path)